# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Dataset:-Imóveis-em-Boston" data-toc-modified-id="Dataset:-Imóveis-em-Boston-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset: Imóveis em Boston</a></div><div class="lev2 toc-item"><a href="#Obtendo-os-dados" data-toc-modified-id="Obtendo-os-dados-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Obtendo os dados</a></div><div class="lev3 toc-item"><a href="#Detetando-um-problema" data-toc-modified-id="Detetando-um-problema-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Detetando um problema</a></div><div class="lev2 toc-item"><a href="#Limpando-e-normalizando-os-dados" data-toc-modified-id="Limpando-e-normalizando-os-dados-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Limpando e normalizando os dados</a></div><div class="lev2 toc-item"><a href="#Salvando-os-arrays-no-folder-data-para-uso-futuro" data-toc-modified-id="Salvando-os-arrays-no-folder-data-para-uso-futuro-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Salvando os arrays no folder data para uso futuro</a></div><div class="lev2 toc-item"><a href="#Regressão-Linear" data-toc-modified-id="Regressão-Linear-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Regressão Linear</a></div><div class="lev2 toc-item"><a href="#Aprendizados" data-toc-modified-id="Aprendizados-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Aprendizados</a></div>

# Dataset: Imóveis em Boston

O objetivo deste notebook é 
- demonstrar uma leitura e análise inicial de um dataset visando
- encontrar eventuais inconsistências e 
- pré processá-lo para tornar mais adaptado ao uso de treinamento de uma rede neural.

Os dados processados são salvos em arquivo para ser lido posteriormente.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython.display import display
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf

## Obtendo os dados

In [2]:
# --------------------------------------------------------------------------------
# Boston Housing Data Set
# -----------------------
# [https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names
# --------------------------------------------------------------------------------
from keras.datasets import boston_housing
(Xtra, ytra), (Xval, yval) = boston_housing.load_data()

Xpd, ypd = pd.DataFrame(Xtra), pd.DataFrame(ytra)
display(Xpd.head())
display(ypd.head())

Using TensorFlow backend.


0     1      2    3      4      5      6       7     8      9     10  \
0  1.23247   0.0   8.14  0.0  0.538  6.142   91.7  3.9769   4.0  307.0  21.0   
1  0.02177  82.5   2.03  0.0  0.415  7.610   15.7  6.2700   2.0  348.0  14.7   
2  4.89822   0.0  18.10  0.0  0.631  4.970  100.0  1.3325  24.0  666.0  20.2   
3  0.03961   0.0   5.19  0.0  0.515  6.037   34.5  5.9853   5.0  224.0  20.2   
4  3.69311   0.0  18.10  0.0  0.713  6.376   88.4  2.5671  24.0  666.0  20.2   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
2  375.52   3.26  
3  396.90   8.01  
4  391.43  14.65

0
0  18.72
1   3.11
2   3.26
3   8.01
4  14.65

### Detetando um problema

Observe que o atributo 12 do dataset é igual ao target (y):

In [3]:
print(Xtra[:5,-1], Xtra.dtype)
print(ytra[:5], ytra.dtype)

[ 18.72   3.11   3.26   8.01  14.65] float64
[ 18.72   3.11   3.26   8.01  14.65] float64


## Limpando e normalizando os dados

Retira-se a coluna 12 do dataset e normaliza-se
os dados para média zero e variância 1 por característica.

In [4]:
Xtra, Xval = Xtra[:,:-1].astype(np.float32), Xval[:,:-1].astype(np.float32)
ytra, yval = ytra.reshape(-1, 1).astype(np.float32), yval.reshape(-1, 1).astype(np.float32)

mean = Xtra.mean(axis=0)
Xtra -= mean
std = Xtra.std(axis=0)
Xtra /= std

Xval -= mean
Xval /= std

m, n = Xtra.shape
print('Shapes:', Xtra.shape, Xval.shape, Xtra.dtype, ytra.shape, yval.shape)
print('Data: min={:.3f}, mean={:.3f}, max={:.3f}, std={:.3f}'.format(Xtra.min(), Xtra.mean(), Xtra.max(), Xtra.std()))
print('Target: min={:.3f}, mean={:.3f}, max={:.3f}, std={:.3f}'.format(ytra.min(), ytra.mean(), ytra.max(), ytra.std()))


Shapes: (404, 12) (102, 12) float32 (404, 1) (102, 1)
Data: min=-3.817, mean=-0.000, max=9.235, std=1.000
Target: min=1.730, mean=12.741, max=37.970, std=7.246


\begin{align*} 
\mathbf{X}_{train} & = \begin{bmatrix}
\longleftarrow & (\mathbf{x}^{(0)})^T & \longrightarrow \\ 
\longleftarrow & (\mathbf{x}^{(1)})^T & \longrightarrow \\ 
  & \vdots  & \\ 
\longleftarrow & (\mathbf{x}^{(m-1)})^T & \longrightarrow 
\end{bmatrix} = \begin{bmatrix}
x_0^{(0)} & x_1^{(0)} & x_2^{(0)} & \ldots & x_{11}^{(0)} \\
x_0^{(1)} & x_1^{(1)} & x_2^{(1)} & \ldots & x_{11}^{(1)} \\
 &  & \vdots  & \\ 
x_0^{(403)} & x_1^{(403)} & x_2^{(403)} & \ldots & x_{11}^{(403)}
\end{bmatrix} \\
\\
\mathbf{y}_{train} & = \begin{bmatrix}
y^{(0)} \\
y^{(1)} \\
\vdots  \\ 
y^{(403)} \\
\end{bmatrix}
\end{align*}


## Salvando os arrays no folder data para uso futuro

Utiliza-se a função `savez` para salvar vários arrays do NumPy num único arquivo, para posterior leitura.

In [5]:
np.savez('../data/boston_housing', Xtra=Xtra, Xval=Xval, ytra=ytra, yval=yval)

## Regressão Linear

Vamos calcular uma regressão linear simples, para termos uma ideia do valor do custo. 

$$ \mathbf{\hat{y}} = \mathbf{X} \cdot \mathbf{W} $$

Utilizamos a *equação normal* para calcular os pesos:

$$  \mathbf{W} =(\mathbf{X}^T \mathbf{X})^{−1}\mathbf{X}^T \mathbf{y} $$

In [6]:
W = np.linalg.inv((Xtra.T).dot(Xtra)).dot(Xtra.T).dot(ytra)

mse_train = np.mean(np.square(np.dot(Xtra, W) - ytra))
mse_valid = np.mean(np.square(np.dot(Xval, W) - yval))

print('MSE train: {:.3f}'.format(mse_train))
print('MSE valid: {:.3f}'.format(mse_valid))

MSE train: 181.229
MSE valid: 181.814


## Aprendizados
